In [72]:
import pyro
import torch

import numpy as np

In [76]:
test = np.zeros(3)
test[2] = 1

In [77]:
test

array([0., 0., 1.])

In [190]:
def model(time_points):
    
    s1_t = pyro.sample("s1_start", pyro.distributions.Uniform(0, 10))
    s2_t = pyro.sample("s2_start", pyro.distributions.Uniform(0, 10))
    s3_t = pyro.sample("s3_start", pyro.distributions.Uniform(0, 10))
    
    r1 = pyro.sample("rate", pyro.distributions.Normal(.5, .01))
    
    t_t = pyro.sample("time_start", pyro.distributions.Exponential(np.sum([s1_t, s2_t, s3_t])))
    
    for t in range(0, time_points):
        
        sample = pyro.sample("sample_{0}".format(str(t)), pyro.distributions.Categorical(
            torch.tensor([s1_t*r1, 
                          s2_t*.1, 
                          s3_t*.9])))
        update = np.zeros(3)
        update[sample] = 1

        s1_t = pyro.sample("s1_{0}".format(str(t)), pyro.distributions.Normal(s1_t + update[0], .00001))
        s2_t = pyro.sample("s2_{0}".format(str(t)), pyro.distributions.Normal(s2_t + update[1], .00001))
        s3_t = pyro.sample("s3_{0}".format(str(t)), pyro.distributions.Normal(s3_t + update[2], .00001))

        t_temp =  pyro.sample("t_{0}_temp".format(str(t)), pyro.distributions.Exponential(np.sum([s1_t, s2_t, s3_t])))
        t_t =  pyro.sample("t_{0}_temp".format(str(t)), pyro.distributions.Normal(t_t + t_temp, .00001))
        
    return [s1_t, s2_t, s3_t, t_t]

In [191]:
model(30)

[tensor(0.5429), tensor(9.0758), tensor(36.5767), tensor(1.0025)]

In [193]:
def model(time_points, data, guess):
    
    s1_t = pyro.sample("s1_start", pyro.distributions.Uniform(0, 10))
    s2_t = pyro.sample("s2_start", pyro.distributions.Uniform(0, 10))
    s3_t = pyro.sample("s3_start", pyro.distributions.Uniform(0, 10))
    
    r1 = pyro.sample("rate", pyro.distributions.Normal(guess, .01))
    
    t_t = pyro.sample("time_start", pyro.distributions.Exponential(np.sum([s1_t, s2_t, s3_t])))
    
    for t in range(0, time_points):
        
        sample = pyro.sample("sample_{0}".format(str(t)), pyro.distributions.Categorical(
            torch.tensor([s1_t*r1, 
                          s2_t*.1, 
                          s3_t*.9])))
        update = np.zeros(3)
        update[sample] = 1
        if t == 500:         
            s1_t = pyro.sample("s1_{0}".format(str(t)), pyro.distributions.Normal(s1_t + update[0], .00001), obs=data[0])
            s2_t = pyro.sample("s2_{0}".format(str(t)), pyro.distributions.Normal(s2_t + update[1], .00001))
            s3_t = pyro.sample("s3_{0}".format(str(t)), pyro.distributions.Normal(s3_t + update[2], .00001))
            t_temp =  pyro.sample("t_{0}_temp".format(str(t)), pyro.distributions.Exponential(np.sum([s1_t, s2_t, s3_t])))
            t_t =  pyro.sample("t_{0}".format(str(t)), pyro.distributions.Normal(t_t + t_temp, .00001), obs=data[1])
        else:
            s1_t = pyro.sample("s1_{0}".format(str(t)), pyro.distributions.Normal(s1_t + update[0], .00001))
            s2_t = pyro.sample("s2_{0}".format(str(t)), pyro.distributions.Normal(s2_t + update[1], .00001))
            s3_t = pyro.sample("s3_{0}".format(str(t)), pyro.distributions.Normal(s3_t + update[2], .00001))
            
            t_temp =  pyro.sample("t_{0}_temp".format(str(t)), pyro.distributions.Exponential(np.sum([s1_t, s2_t, s3_t])))
            t_t =  pyro.sample("t_{0}".format(str(t)), pyro.distributions.Normal(t_t + t_temp, .00001))
        
    return [s1_t, s2_t, s3_t, t_t]

In [170]:
def guide(time_points, data):
    
    s1_t = pyro.sample("s1_start", pyro.distributions.Uniform(0, 10))
    s2_t = pyro.sample("s2_start", pyro.distributions.Uniform(0, 10))
    s3_t = pyro.sample("s3_start", pyro.distributions.Uniform(0, 10))
    
    r1 = pyro.sample("rate", pyro.distributions.Normal(.5, .01))
    t_t = pyro.sample("time_start", pyro.distributions.Exponential(np.sum([s1_t, s2_t, s3_t])))
    for t in range(0, time_points):
        
        sample = pyro.sample("sample_{0}".format(str(t)), pyro.distributions.Categorical(
            torch.tensor([s1_t*r1, 
                          s2_t*.1, 
                          s3_t*.9])))
        update = np.zeros(3)
        update[sample] = 1
                 
        s1_t = pyro.sample("s1_{0}".format(str(t)), pyro.distributions.Normal(s1_t + update[0], .00001))
        s2_t = pyro.sample("s2_{0}".format(str(t)), pyro.distributions.Normal(s2_t + update[1], .00001))
        s3_t = pyro.sample("s3_{0}".format(str(t)), pyro.distributions.Normal(s3_t + update[2], .00001))
        t_temp =  pyro.sample("t_{0}_temp".format(str(t)), pyro.distributions.Exponential(np.sum([s1_t, s2_t, s3_t])))
        t_t =  pyro.sample("t_{0}".format(str(t)), pyro.distributions.Normal(t_t + t_temp, .00001))
    
    return [s1_t,s2_t,s3_t]

In [162]:
model(1000, [100.0,4.0])

/home/dkohler/anaconda3/lib/python3.9/site-packages/pyro/primitives.py:137: RuntimeWarning: trying to observe a value outside of inference at s1_500
  warnings.warn(
/home/dkohler/anaconda3/lib/python3.9/site-packages/pyro/primitives.py:137: RuntimeWarning: trying to observe a value outside of inference at t_500_temp
  warnings.warn(


[tensor(141.9999), tensor(4.0959), tensor(871.6047), tensor(4.6760)]

In [174]:
# set up the optimizer
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = pyro.optim.Adam(adam_params)

# setup the inference algorithm
svi = pyro.infer.SVI(model, guide, optimizer, loss=pyro.infer.Trace_ELBO())

n_steps = 5000
# do gradient steps
for step in range(n_steps):
    svi.step(1000, torch.tensor([100.0,4.0]))

KeyboardInterrupt: 

In [195]:
nuts_kernel = pyro.infer.NUTS(model)
mcmc = pyro.infer.MCMC(nuts_kernel, num_samples=500)
mcmc.run(30, torch.tensor([10.0, 1.0]), torch.tensor(.5))
samples = mcmc.get_samples()

Warmup:   0%|                                                                                    | 0/1000 [00:00, ?it/s]

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.
Trace Shapes:  
 Param Sites:  
Sample Sites:  
s1_start dist |
        value |
s2_start dist |
        value |
s3_start dist |
        value |
    rate dist |
        value |